# NoteBook To Check Point or Polygon Inside Polygon

The objective of this notebook is to search a set of points or polygons for spatial overlap with a largerset

By Cascsde Tuholske 2019-01-17

#### Notes
1. Points should be dialated to circles to prevent one falling just outside the larger polygon set (see Lusaka)

### Depedencies

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import mapping
from shapely.geometry import Polygon
from shapely.geometry import shape
import ast
from shapely.geometry import mapping
import rasterio
import copy

### Functions

In [2]:
def load_points (file):
    """ This function loads a csv 
    of points and turns it into shapely points"""
    df = pd.read_csv(file)

    # creating a geometry column 
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

    # Coordinate reference system : WGS84
    crs = {'init': 'epsg:4326'}

    # Creating a Geographic data frame 
    point_gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
    
    return point_gdf

In [3]:
def point_buffer(gpd_df, raduis):
    "Function to make a shapely polygon buffer around a point"
   
    # AGU 2018-12-04 - radius set to ~250m at the equator 
    #radius = 250*1/(111*1000)

    # new_gpd_df = gpd.GeoDataFrame()
    gpd_df.rename(columns={'geometry':'old_geom'}, inplace=True)
    
    arr = []
    
    for point in gpd_df['old_geom']:
        buffer = point.buffer(radius)
        arr.append((buffer))
    
    gpd_df['geometry'] = arr
    
    return gpd_df

In [4]:
def poly_point (poly, point):
    """
    This function will check if points are inside polygons if given two gpd dataframes with points and polygons
    Returns point ids, point geometry, polygon index # and polygon geometry
    """
    
    out_arr = [] #return an array <<< ---------------- ASK RYAN IF BETTER DO USE DICT 
    
    for index_point, row_point in point.iterrows():
        for index_poly, row_poly in poly.iterrows():
            if row_point['geometry'].within(row_poly['geometry']):
                point_id = row_point['id']
                point_geom = mapping(row_point['geometry']) # makes a dict w/ keys : type and cood
                poly_id = index_poly
                poly_geom = mapping(row_poly['geometry']) # makes a dict w/ keys : type and cood
                
                out_arr.append((point_id, 
                                point_geom, 
                                poly_id, 
                                poly_geom))

    return out_arr

# Note 2018-11-30 update arr to gpd_df 

In [5]:
def poly_overlap (point_buffer, poly_raster):
    """
    This function will check if point buffers intersect with polygons 
    if given two gpd dataframes with point buffers and polygons
    Returns point ids, point geometry, polygon index #, and polygon geometry in a geopandas DF.
    It goes faster if smaller list goes first
    
    Change TOWN & CITY ARRA
    """
    
    # make arrays to fill
    country_arr = []
    osm_name_arr = []
    osm_type_arr = []
    lat_arr = []
    lon_arr = []
    osm_id_arr = [] 
    FID_arr = [] 
    poly_geom_arr = []
    #point_geom_arr = []
    
    for index_point_buffer, row_point_buffer in point_buffer.iterrows():
        for index_poly_raster, row_poly_raster in poly_raster.iterrows():
            if row_point_buffer['geometry'].intersects(row_poly_raster['geometry']):
                country = row_point_buffer['country']
                osm_name = row_point_buffer['osm_name'] 
                osm_type = row_point_buffer['osm_type']
                lat = row_point_buffer['lat']
                lon = row_point_buffer['lon']
                osm_id = row_point_buffer['osm_id']
                poly_id = row_poly_raster['FID']
                poly_geom = shape(mapping(row_poly_raster['geometry'])) # make polygon
                #point_geom = row_point_buffer['old_geom']

                country_arr.append((country))
                osm_name_arr.append((osm_name))
                osm_type_arr.append((osm_type))
                lat_arr.append((lat))
                lon_arr.append((lon))
                osm_id_arr.append((osm_id))
                FID_arr.append((poly_id))
                poly_geom_arr.append((poly_geom))
                #point_geom_arr.append((point_geom))
    
    # put results into a geopandas df
    new_gpd_df = gpd.GeoDataFrame()
    new_gpd_df['osm_id'] = osm_id_arr
    new_gpd_df['FID'] = FID_arr
    new_gpd_df['geometry'] = poly_geom_arr
    new_gpd_df['country'] = country_arr           
    new_gpd_df['osm_name'] = osm_name_arr
    new_gpd_df['osm_type'] = osm_type_arr
    new_gpd_df['lat'] = lat_arr
    new_gpd_df['lon'] = lon_arr
    #new_gpd_df['point_geom'] = point_geom_arr
    
    
    return new_gpd_df

# Note 2018-11-30 update arr to gpd_df 
# Note 2019-04-30 updated to use osm_name instead of city/town for the OSM name

### Load Data

In [6]:
import os
os.getcwd()

'/tana-crunch/cascade/projects/Pop-ERL/notebooks/jupyter/ERL19'

In [7]:
# File Paths
temp_data = '../../../temp_data/ERL19/'
erl_v2_data  = '../../../temp_data/ERL19v2/'

In [8]:
# Load Points
points = gpd.read_file(erl_v2_data+'20190430_osm_All.shp')
print(len(points))


9813


In [9]:

# Load Polygons
polygons = gpd.read_file(temp_data+'GHS_POP_GPW42000_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300_C.shp')
print(len(polygons))
    
#Out File Name
out = 'GHS_POP_GPW42000_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300_C_polypoints'

15281


In [10]:
polygons.head()

,FID,geometry
0,30000,POLYGON ((3507405.452556469 -32970.53813151084...
1,30001,POLYGON ((3582405.452556469 -32970.53813151084...
2,30002,POLYGON ((3593405.452556469 -32970.53813151084...
3,30003,POLYGON ((3644405.452556469 -32970.53813151084...
4,30004,POLYGON ((3808405.452556469 -32970.53813151084...


#### Change CRS if needed

For GHS-Pop, tranform points into ESRI:54009 for polysearch

In [11]:
polygons.crs

{'proj': 'moll',
 'lon_0': 0,
 'x_0': 0,
 'y_0': 0,
 'ellps': 'WGS84',
 'units': 'm',
 'no_defs': True}

In [12]:
points.crs

{'init': 'epsg:4326'}

In [13]:
# For GHS CRS

crs = {'ellps': 'WGS84',
 'lon_0': 0,
 'no_defs': True,
 'proj': 'moll',
 'units': 'm',
 'x_0': 0,
 'y_0': 0}

In [14]:
polygons.crs = crs

In [15]:
polygons.crs

{'ellps': 'WGS84',
 'lon_0': 0,
 'no_defs': True,
 'proj': 'moll',
 'units': 'm',
 'x_0': 0,
 'y_0': 0}

In [16]:
points.head()

,country,osm_id,osm_name,lat,lon,osm_type,geometry
0,Algeria,89369215,Tamanrasset,22.785454,5.532446,town,POINT (5.5324465 22.7854543)
1,Algeria,89980948,In Salah,27.195033,2.482613,town,POINT (2.4826132 27.1950331)
2,Algeria,252600742,Boumerdès,36.758882,3.470596,town,POINT (3.470596 36.758882)
3,Algeria,253167052,Thenia,36.724986,3.556935,town,POINT (3.556935 36.724986)
4,Algeria,253167208,Zemmouri,36.786406,3.601221,town,POINT (3.6012209 36.7864064)


In [17]:
points = points.to_crs(polygons.crs)

In [18]:
points.head()

,country,osm_id,osm_name,lat,lon,osm_type,geometry
0,Algeria,89369215,Tamanrasset,22.785454,5.532446,town,POINT (527311.6502082234 2788716.35127626)
1,Algeria,89980948,In Salah,27.195033,2.482613,town,POINT (231414.6222862774 3313817.730679021)
2,Algeria,252600742,Boumerdès,36.758882,3.470596,town,POINT (303122.9375980464 4424001.504927466)
3,Algeria,253167052,Thenia,36.724986,3.556935,town,POINT (310749.4720741886 4420148.212193361)
4,Algeria,253167208,Zemmouri,36.786406,3.601221,town,POINT (314461.2710128424 4427130.004381498)


#### Buffer Points

In [19]:
# AGU 2018-12-04 - radius set to ~250m at the equator 
# 2019 05 01 --> Buffer should 

# radius = 250*1/(111*1000)
radius = 250

In [20]:
osm_buffer_gdf = point_buffer(points, radius)

In [21]:
osm_buffer_gdf.head(6)

,country,osm_id,osm_name,lat,lon,osm_type,old_geom,geometry
0,Algeria,89369215,Tamanrasset,22.785454,5.532446,town,POINT (527311.6502082234 2788716.35127626),"POLYGON ((527561.6502082234 2788716.35127626, ..."
1,Algeria,89980948,In Salah,27.195033,2.482613,town,POINT (231414.6222862774 3313817.730679021),"POLYGON ((231664.6222862774 3313817.730679021,..."
2,Algeria,252600742,Boumerdès,36.758882,3.470596,town,POINT (303122.9375980464 4424001.504927466),"POLYGON ((303372.9375980464 4424001.504927466,..."
3,Algeria,253167052,Thenia,36.724986,3.556935,town,POINT (310749.4720741886 4420148.212193361),"POLYGON ((310999.4720741886 4420148.212193361,..."
4,Algeria,253167208,Zemmouri,36.786406,3.601221,town,POINT (314461.2710128424 4427130.004381498),"POLYGON ((314711.2710128424 4427130.004381498,..."
5,Algeria,253291208,Lakhdaria,36.563944,3.596907,town,POINT (314651.9864841976 4401832.281901533),"POLYGON ((314901.9864841976 4401832.281901533,..."


In [22]:
# drop point geom and keep buffered

osm_buffer_gdf = osm_buffer_gdf.drop(['old_geom'], axis=1)
osm_buffer_gdf.head(6)

,country,osm_id,osm_name,lat,lon,osm_type,geometry
0,Algeria,89369215,Tamanrasset,22.785454,5.532446,town,"POLYGON ((527561.6502082234 2788716.35127626, ..."
1,Algeria,89980948,In Salah,27.195033,2.482613,town,"POLYGON ((231664.6222862774 3313817.730679021,..."
2,Algeria,252600742,Boumerdès,36.758882,3.470596,town,"POLYGON ((303372.9375980464 4424001.504927466,..."
3,Algeria,253167052,Thenia,36.724986,3.556935,town,"POLYGON ((310999.4720741886 4420148.212193361,..."
4,Algeria,253167208,Zemmouri,36.786406,3.601221,town,"POLYGON ((314711.2710128424 4427130.004381498,..."
5,Algeria,253291208,Lakhdaria,36.563944,3.596907,town,"POLYGON ((314901.9864841976 4401832.281901533,..."


### Intersection

In [ ]:
# run poly_overlap

import time
checkpoint = time.time()

poly_overlap_out = poly_overlap(osm_buffer_gdf, polygons)

print("elapsed time is: {}s".format(time.time()-checkpoint))
print(len(poly_overlap_out))

In [ ]:
print(len(poly_overlap_out))

In [ ]:
poly_overlap_out.tail(20)

In [ ]:
poly_overlap_out.to_file(erl_v2_data+out+'.shp', driver='ESRI Shapefile')